# Pulizia crew.csv

In [6]:
import pandas as pd
import os
from unidecode import unidecode

In [7]:
# Risali alla cartella "Solutions"
script_dir = os.getcwd()
solutions_dir = os.path.abspath(os.path.join(script_dir, ".."))

# Imposta "Data" come directory di lavoro
data_dir = os.path.join(solutions_dir, "Data")
os.chdir(data_dir)  # Cambia la directory di lavoro

# Ora puoi riferirti direttamente ai file dentro "Data"
file_path = os.path.join("main_data", "crew.csv")

# Costruisce il percorso relativo al file CSV pulito
clean_path = os.path.join(data_dir, "clean_data")

# Controlla che il file esista prima di caricarlo
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Il file non esiste: {file_path}")

# Carica il CSV in un DataFrame
df = pd.read_csv(file_path)

In [8]:
# Funzione per tradurre il nome in caratteri latini 
def transliterate_name(name):
    if isinstance(name, str):  # Controlla che sia una stringa
        return unidecode(name).strip()  # Traduci e rimuovi spazi extra
    return name  # Se non è stringa, lascialo così

# Applica la funzione alla colonna dei nomi e dei ruoli
df['name'] = df['name'].apply(transliterate_name)

df['role'] = df['role'].fillna("Unknown")
df['name'] = df['name'].fillna("Unknown")

In [9]:
# Salva il file pulito
cleaned_csv_path = os.path.join(clean_path, "crew_cleaned.csv")
df.to_csv(cleaned_csv_path, index=False, encoding="utf-8")

# Caricamento file csv pulito nel database con PostgreSQL


In [1]:
import psycopg2
from sqlalchemy import create_engine,text

In [3]:
#  Parametri di connessione PostgreSQL
DB_NAME = "ium_database"
DB_USER = "postgres"
DB_PASSWORD = "password"
DB_HOST = "localhost"
DB_PORT = "5432"

try:
    conn = psycopg2.connect(dbname="postgres", user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT) # Crea una connessione al database usando psycopg2.connect
    conn.autocommit = True # Conferma automaticamente ogni operazione evitando la conferma manuale dopo ogni query
    cur = conn.cursor() # Crea un oggetto cur che permette di eseguire query SQL sul database

    cur.execute(f"CREATE DATABASE {DB_NAME};")# Esegue il comando per creare il database con il nome specificato
    print(f"Database '{DB_NAME}' creato con successo.")
    
except psycopg2.errors.DuplicateDatabase:
    print(f"Il database '{DB_NAME}' esiste già.")

Il database 'ium_database' esiste già.


In [12]:
# Crea un oggetto di connessione a un database PostgreSQL utilizzando SQLAlchemy
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [13]:
# Query SQL per creare la tabella
create_table_query = text("""
    CREATE TABLE IF NOT EXISTS crew_data (
        id INT,
        role TEXT,
        name TEXT
    );
""")

# Eseguire la query
with engine.connect() as connection: # Apre una connessione al database usando SQLAlchemy
    connection.execute(create_table_query) # Esegue il comando per la creazione della 
    connection.commit() # Salva le modifiche

In [14]:
# Legge il CSV pulito
df = pd.read_csv(cleaned_csv_path)

# Carica i dati nella tabella PostgreSQL
df.to_sql("crew_data", engine, if_exists="replace", index=False) # Esporta il dataframe nel database, sovrascrive e aggiunge i nuovi dati alla tabella esistente

183